In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import calculate_bollinger_bands, calculate_rsi
from utilities import print_title, print_label, print_footer

In [2]:
# Import libraries for data analysis and visualization
import pandas as pd
import numpy as np
import yfinance as yf # Yahoo Finance data retrieval

# Import libraries for signal processing and peak detection
from scipy.signal import find_peaks

# Import libraries for plotting and visualization
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh') # Use Bokeh as the backend for Holoviews

# Import libraries for data storage and retrieval
from pathlib import Path

In [3]:
# Set display options for Pandas (optional, but often helpful)
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Suppress scientific notation

### Data Collection
___

In [9]:
# Fetch S&P 500 tickers

#Get tickers from wikipedia
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_numpy()

# Filter out Class B shares that have a '.B' in the ticker name
# Class B shares are typically held by company insiders and have different voting rights than Class A shares
sp500_tickers = [ticker for ticker in sp500_tickers if '.B' not in ticker]

# print(f"Initial total S&P 500 tickers: {len(sp500_tickers)}")
print_title("Initial total S&P 500 tickers: " + str(len(sp500_tickers)), closed_corners=True)

╔═══════════════════════════════════════════════════════════════╗
║              Initial total S&P 500 tickers: 501               ║
╚═══════════════════════════════════════════════════════════════╝


In [18]:
#Define dates
start_date = '2000-01-01'
end_date = '2024-10-14'

# Download historical prices
data = yf.download(sp500_tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  501 of 501 completed


### Data Preprocessing
___

In [17]:
data.shape
data

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03 00:00:00+00:00,43.463,0.844,NaN,NaN,8.288,1.278,NaN,16.275,28.215,6.307,...,NaN,11.353,NaN,6.848,18.036,NaN,4.634,NaN,25.028,NaN
2000-01-04 00:00:00+00:00,40.143,0.773,NaN,NaN,8.051,1.271,NaN,14.909,26.787,6.242,...,NaN,10.926,NaN,7.006,17.690,NaN,4.541,NaN,24.667,NaN
2000-01-05 00:00:00+00:00,37.653,0.784,NaN,NaN,8.037,1.389,NaN,15.204,27.178,6.143,...,NaN,11.505,NaN,7.276,18.655,NaN,4.564,NaN,25.139,NaN
2000-01-06 00:00:00+00:00,36.219,0.716,NaN,NaN,8.318,1.375,NaN,15.328,26.435,6.176,...,NaN,12.043,NaN,7.209,19.619,NaN,4.526,NaN,23.778,NaN
2000-01-07 00:00:00+00:00,39.237,0.750,NaN,NaN,8.407,1.451,NaN,16.073,27.178,6.274,...,NaN,11.647,NaN,7.209,19.562,NaN,4.425,NaN,23.514,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-07 00:00:00+00:00,143.890,221.690,192.076,130.230,112.828,107.790,355.354,487.300,226.510,58.860,...,287.990,32.850,106.410,63.010,125.370,134.030,135.880,102.280,365.240,187.280
2024-10-08 00:00:00+00:00,144.610,225.770,191.481,132.500,114.092,109.910,358.919,496.240,229.400,57.550,...,292.940,33.110,102.880,62.930,122.040,134.200,134.720,101.770,368.970,187.370
2024-10-09 00:00:00+00:00,144.800,229.540,193.207,135.490,115.406,113.090,363.600,494.080,233.240,56.970,...,298.050,33.060,104.150,62.580,122.090,136.290,134.760,103.460,373.500,190.170


In [ ]:
# Download historical prices
# data = yf.download(sp500_tickers, start=start_date, end=end_date)['Adj Close']

# Fill NaN values with 0
data.fillna(0, inplace=True)

# Check if data was downloaded for all tickers
# print(f"Successfully downloaded tickers: {len(data.columns)} out of {len(sp500_tickers)}")

In [ ]:
# Display data Frame
data.head()

In [ ]:
# Display tickers that were successfully downloaded
downloaded_tickers = data.columns.get_level_values(0).unique()
print(f"Successfully downloaded tickers: {len(downloaded_tickers)} out of {len(sp500_tickers)}")

In [8]:
# Calculate various indicators
volatility = data.pct_change()

# Handle inf values in the volatility DataFrame
volatility.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill missing values
volatility.ffill(inplace=True)
volatility.bfill(inplace=True)

rsi = data.apply(calculate_rsi)
sma_50 = data.rolling(window=50).mean()
sma_100 = data.rolling(window=100).mean()
sma_200 = data.rolling(window=200).mean()
upper_band, lower_band = calculate_bollinger_bands(data)
support = data.rolling(window=50).min()
resistance = data.rolling(window=50).max()


In [9]:
# Align data lengths by filling forward and backward
volatility.ffill(inplace=True)
volatility.bfill(inplace=True)
rsi.ffill(inplace=True)
rsi.bfill(inplace=True)
sma_50.ffill(inplace=True)
sma_50.bfill(inplace=True)
sma_100.ffill(inplace=True)
sma_100.bfill(inplace=True)
sma_200.ffill(inplace=True)
sma_200.bfill(inplace=True)
upper_band.ffill(inplace=True)
upper_band.bfill(inplace=True)
lower_band.ffill(inplace=True)
lower_band.bfill(inplace=True)
support.ffill(inplace=True)
support.bfill(inplace=True)
resistance.ffill(inplace=True)
resistance.bfill(inplace=True)

In [ ]:
print(f"Volatility shape: {volatility.shape}")
print(volatility.head())

print(f"RSI shape: {rsi.shape}")
print(rsi.head())

print(f"SMA_50 shape: {sma_50.shape}")
print(sma_50.head())

print(f"SMA_100 shape: {sma_100.shape}")
print(sma_100.head())

print(f"SMA_200 shape: {sma_200.shape}")
print(sma_200.head())

print(f"Upper Band shape: {upper_band.shape}")
print(upper_band.head())

print(f"Lower Band shape: {lower_band.shape}")
print(lower_band.head())

print(f"Support shape: {support.shape}")
print(support.head())

print(f"Resistance shape: {resistance.shape}")
print(resistance.head())

In [ ]:
# Compile indicators into a single DataFrame
length = len(data) * len(data.columns)
indicators_df = pd.DataFrame({
    'Date': np.repeat(data.index, len(data.columns)),
    'Ticker': np.tile(data.columns, len(data)),
    'Adjusted Close': data.values.flatten(),
    'Volatility': volatility.values.flatten(),
    'RSI': rsi.values.flatten(),
    'SMA_50': sma_50.values.flatten(),
    'SMA_100': sma_100.values.flatten(),
    'SMA_200': sma_200.values.flatten(),
    'Upper Band': upper_band.values.flatten(),
    'Lower Band': lower_band.values.flatten(),
    'Support': support.values.flatten(),
    'Resistance': resistance.values.flatten(),
})

# Drop NaN values
indicators_df.dropna(subset=['Adjusted Close', 'Volatility', 'RSI', 'SMA_50', 'SMA_100', 'SMA_200', 'Upper Band', 'Lower Band', 'Support', 'Resistance'], inplace=True)

# Display DataFrame
print(indicators_df.head())

In [ ]:
indicators_df.head()

In [ ]:
# Plot RSI and Bollinger Bands for a sample stock (e.g., AAPL)
ticker_sample = 'AAPL'
plt.figure(figsize=(14, 7))
plt.subplot(2, 1, 1)
plt.plot(data[ticker_sample], label=f'{ticker_sample} Price')
plt.plot(sma_50[ticker_sample], label='50-Day SMA', linestyle='--')
plt.plot(upper_band[ticker_sample], label='Upper Bollinger Band', linestyle='--', color='orange')
plt.plot(lower_band[ticker_sample], label='Lower Bollinger Band', linestyle='--', color='orange')
plt.fill_between(data.index, lower_band[ticker_sample], upper_band[ticker_sample], color='lightgray')
plt.legend()
plt.title(f'{ticker_sample} Price with Bollinger Bands')
plt.subplot(2, 1, 2)
plt.plot(rsi[ticker_sample], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--')
plt.axhline(30, color='green', linestyle='--')
plt.legend()
plt.title(f'{ticker_sample} RSI')
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for plotting
ticker_sample = 'AAPL'
data_sample = data[[ticker_sample]].copy()
data_sample['SMA_50'] = sma_50[ticker_sample]
data_sample['SMA_100'] = sma_100[ticker_sample]
data_sample['SMA_200'] = sma_200[ticker_sample]
data_sample['RSI'] = rsi[ticker_sample]
data_sample['Support'] = support[ticker_sample]
data_sample['Resistance'] = resistance[ticker_sample]

# Plot
price_plot = data_sample.hvplot.line(y=[ticker_sample, 'SMA_50', 'SMA_100', 'SMA_200'], title=f'{ticker_sample} Price with Bollinger Bands, SMA, and Resistance Levels', ylabel='Price')
support_resistance_plot = data_sample.hvplot.line(y=['Support', 'Resistance'], line_dash='dashed', color=['green', 'red'])
rsi_plot = data_sample.hvplot.line(y='RSI', title=f'{ticker_sample} RSI', ylabel='RSI').opts(yformatter='%.0f') * hv.HLine(70).opts(color='red', line_dash='dashed') * hv.HLine(30).opts(color='green', line_dash='dashed')

(price_plot * support_resistance_plot + rsi_plot).cols(1)
